
<center> <H1> Classes for Ganule distribution analysis </H1> </center>



                        *Code written by Timo Rey. Laboratory of Experimental Biophysics, EPFL*

                                            *Created during revisions in 2019/20*

#### Aims:
    Provide a class for simulation of randomly placed particles within a container.
    Provide a class to hold observed values of particles within a container.

The simulation of random-placing of particles was heavily inspired by a publication in *Science* No. 351, in 2016
doi:10.1126/science.aaa8714 by **R. Jajoo et al.**

#### Use:
    This code can be called from other (jupyter notebook) python scripts to use the classes in further analysis.
#### Libraries:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns

# Class "RandomSimulation"

    Instance_name = RandomSimulation(
    r'Name',                                             # Name -> new ID for pairing with observed data
    Dimension,                                           # Box dimensions (1D, 2D, 3D) - Only 1D so far.
    Box,                                                 # Box size [length, width, depth]
    Particles,                                           # Number of particles / Granules per box
    Resolution,                                          # Excluded volume around each particle)

In [3]:
class RandomSimulation:
    """ Create a new instance, defined by several parameters """

                        #{0}  #{1}   #{2}        #{3}             #{4}
    def __init__(self, Name,  Box, Particles, Resolution=0.14, Dimension=1):

        self.Name             = Name
        self.Box              = Box
        self.Particles        = Particles
        self.Resolution       = Resolution
        self.Dimension        = Dimension #so far only 1D
 
        self.all_runs         = []
        self.norm_positions   = []
        self.distances        = []
        
        #print("Simulation '{0}' was successfully created".format(self.Name))   

    def __str__(self): #can call this with print(self)
        return """Simulation:
 Name: {0} \n Box-size: {1} [in um] \n Number of particles per box: {2}
 Excluded volume constraint: {3} [in um]""".format(self.Name, self.Box, self.Particles, self.Resolution)

    
# *************************************************************************************************************************        
    """ Generate simulated random data-set """  
        
# generate 1D random placements:
    def random_placement_1D(self):
        position               = []
        for i in range(self.Particles):                         # generate x-number of particles simultaneously
            position.append(self.Box[0]*np.random.rand())       # placing them along the length of the box.
        return position

# check distances to end-points of the box (poles) are larger than the resolution limit:
    def excluded_Poles(self, positions):
        check                  = []
        
        for i in positions:                                     # for each generated position
            if i < (self.Resolution/2) or (self.Box[0]-i) < (self.Resolution/2):
                check.append(False)                             # verify it is not too close to the edges (1/2 excluded volume)
            else:
                check.append(True)

        if np.array(check).all() == False:                      # check none of the granules dissatisfy the conditions
            #print("some are smaller")
            is_True = False
        else:
            #print("all are bigger")
            is_True = True            
        return is_True


# verify excluded volume:
    def excluded_Granules(self, positions):
        distances = []
        # find all distances to all other granules:  
        for i in positions:                                    # for all granule-positions
            for x in positions:                                # subtract all other positions
                distances.append(abs(i-x))                     # and add to list

        # check all distances are bigger than the excluded volume / resolution limit:
        check = []
        for i in distances:
            if i == 0:                                         # to remove the self-overlap
                pass
            elif i < self.Resolution:
                check.append(False)
            else:
                check.append(True)

        if np.array(check).all() == False:                      # check none of the granules dissatisfy the conditions
            is_True = False
        else:
            is_True = True            
        return is_True

# run the simulation for this instance of self:
    def run_simulation(self, iterations = 200):
        max_trials_per_run        = 10000                      # to allow a maximum number of trials per data-set before raises error
        self.all_runs             = []                         # can hash this if want to accumulate simulation runs
        i = 0
        failed = 0
        
        while i < iterations:                                  # run for number of iterations
            run = self.random_placement_1D()                   # 1 run of random placement per iteration

            if self.excluded_Poles(run):                       # check whether pole-exclusion is satisfied
                if self.excluded_Granules(run):                # check whether granule-exclusion is satisfied
                    self.all_runs.append(run)                  # only if both: add this run to all_runs list
                    i += 1                                     # and go to next iteration
                else:                                          # else add failed run.
                    failed += 1
            else:
                failed += 1
            
            if failed < max_trials_per_run:                    # if number of failed runs exceeds limit, raise error.
                pass
            else:
                print("""\tERROR ERROR ERROR:
                in data-set %s, run %r you tried %r times to satisfy excluded-volume restrictions.
                This is too many.
                \tERROR ERROR ERROR""" %(self.Name, i, max_trials_per_run))
        #print("\t%s ran for %r iterations" %(self.Name, i))

    
# *************************************************************************************************************************        
    """ Normalise positions and compute particle distances """  

# normalise particle-positions by box-length:
    def normalise_positions(self):
        self.norm_positions      = []
        
        for i in range(len(self.all_runs)):                    # all_runs contains all particle positions for simulations
            self.norm_positions.append(self.all_runs[i]/self.Box[0])

# calculate absolute or normalised distances between granules:
    def particle_distances(self, absolute = True):             # by default, compute aboslute distances between particles
        self.distances           = []
        if self.Particles == 1:
            print("There is only 1 granule - cannot calculate distances.")
            
        else:
            if absolute:
                particles        = self.all_runs       # to choose whether want to analyse aboslute or relative distances
            else:
                particles        = self.norm_positions
                
            for (i,val) in enumerate(particles):

                val.sort()                             # sort particles according to position

                distances        = []
                l = 1
                while l < self.Particles:              # we count # granules from 1
                    distance     = abs(val[l]-val[l-1])# python counts from 0 (e.g. 2nd granule - 1st granule = val[1]-val[0])
                    distances.append(distance)
                    l            += 1

                self.distances.append(distances)

# Calculate nearest neighbour distances to particles for every pole:
    def distance_fromPole(self):
        self.pole_1 = []
        self.pole_2 = []

        for i,v in enumerate(self.all_runs):
            # find particle that lies closest from each pole:
            deltaPole_1 = min(self.all_runs[i])
            deltaPole_2 = min(abs(self.Box[0]-self.all_runs[i]))

            self.pole_1.append(deltaPole_1)
            self.pole_2.append(deltaPole_2)
        
# *************************************************************************************************************************        
    """ Create figures & plots """   

# show one or more, random or specific examples of generated positions:
    def show_example(self, random=True):
        data    = self.all_runs
        length  = self.Box[0]
        y       = np.ones(self.Particles)

        plt.figure()
        plt.title("this is an example of: " + self.Name)
        # plot a line with the length of the mitochondrion:
        plt.plot([0,length], [1,1], c='blue', alpha=0.5, linewidth = 3)
        
        # choose a random simulation to show:
        if random == True:
            i = np.random.randint(0,(len(self.all_runs)))
        else:
            i = random
        # plot the simulated granules on top of the line
        plt.errorbar(data[i], y, c='red', xerr=self.Resolution/2, fmt='o', markersize=2, linewidth = 5)
        # note: choose a marker size not bigger than the excluded volume to show it.


# for a particular mitochondrion, plot histogram of simulated particle distributions from all iterations:
    def plot_distribution(self, data_type = 'absolute', c='blue'):
        if data_type == 'absolute':
            data   = self.all_runs

        elif data_type == 'normalised':    
            data   = self.norm_positions   = []

        elif data_type == 'distances':
            data   = self.distances

        plt.figure()
        plt.title( "Simulated particle positions of: " + self.Name + " after " + str(len(self.all_runs)) + " iterations")
        data   = np.reshape(data, -1)
        sns.distplot(data, kde=False)


# Class "SingleObservation"

    Instance_name = RandomSimulation(
    r'Name',                                             # Name -> new ID for pairing with simulation
    Dimension,                                           # Box dimensions (1D, 2D, 3D) - Only 1D so far.
    Box,                                                 # Observed size of mitochondria [length, width, depth]
    Particles,                                           # Number of particles / Granules per box
    Positions,                                           # Observed granule positions
    Resolution,                                          # Excluded volume around each particle
    Image_ID, Parent_Name,                               # original name of mitochondrion & the acquisition for back-tracing)

In [4]:
class SingleObservation:
    """ Create a new instance, defined by several parameters """

                        #{0}  #{1}    #{2}       #{3}       #{4}          #{5}         #{6}
    def __init__(self, Name,  Box, Particles, Positions, Image_ID, Parent_Name, Resolution=0.14):        

        self.Name           = Name
        self.Box            = Box
        self.Particles      = Particles
        self.Positions      = Positions
        self.Image_ID       = Image_ID
        self.Parent_Name    = Parent_Name
        self.Resolution     = Resolution #default is approximated resolution limit of SIM

        self.norm_positions = []
        self.distances      = []
        self.NN             = []
        #print("Observation object '{0}' was successfully instantiated".format(self.Name))
        
    def __str__(self): #can call this with print(self)
        return """Observation:
 Name: {0} \n Mitochondrial length: {1} [in um] \n With {2} granules at: {3} \n From Image: {4}
 With excluded volume constraint: {5} [in um]""".format(self.Name, self.Box, self.Particles, self.Positions, self.Resolution)


# *************************************************************************************************************************        
    """ Normalise positions and compute particle distances """  

# normalise granule-positions by box-length:
    def normalise_positions(self):
        self.norm_positions = []
        
        for i in range(self.Particles):
            self.norm_positions.append(self.Positions[i]/self.Box[0])

# Calculate absolute or normalised distances between granules:
    def particle_distances(self, absolute = True):             # by default, compute aboslute distances between particles
        self.distances           = []
        if self.Particles == 1:
            print("There is only 1 granule - cannot calculate distances.")
        else:
            if absolute:                                       # choose whether to analyse aboslute or relative distances
                particles        = self.Positions
            else:
                particles        = self.norm_positions
            
            particles.sort()                                   # bring particles into order
            distances        = []
            l = 1
            while l < self.Particles:
                distance     = abs(particles[l]-particles[l-1])
                distances.append(distance)
                l            += 1
                
            self.distances.append(distances)

# Calculate Nearest Neighbour distance for every particle:
    def particle_NN(self):
        self.NN              = []
        if self.Particles <= 1:
            print("There is 1 or fewer granules - cannot calculate NN-distances.")
        else:
            particles        = self.Positions      # make list with all positions
            all_dist = []
            for i,v in enumerate(particles):       # for every particle
                mylist = []
                for i2,v2 in enumerate(particles): # calculate all distances
                    if np.abs(v-v2) > 0:           # exclude 0-distance
                        mylist.append(np.abs(v-v2))# add each distance to list
                        
                all_dist.append(np.min(mylist))    # keep smallest distance from list, for this particle

            self.NN = list(set(all_dist))          # remove duplicates

# Calculate nearest neighbour distances to particles for every pole:
    def distance_fromPole(self):
        self.pole_1 = []
        self.pole_2 = []
        
        # find particle that lies closest to pole 1:
        deltaPole_1 = min(self.Positions)
        self.pole_1.append(deltaPole_1)
        
        # find particle that lies closest to pole 2:
        deltaPole_2 = min(abs(self.Box[0]-self.Positions))
        self.pole_2.append(deltaPole_2)   

# *************************************************************************************************************************        
    """ Create figures & plots """

# show one or more, random or specific examples of generated positions:
    def show_Granules(self):
        length  = self.Box[0]
        y       = np.ones(len(self.Positions))
        data    = self.Positions
        
        plt.figure()
        plt.title(str(self.Particles) + " granules in " + self.Name)
        # plot a line with the length of the mitochondrion:
        plt.plot([0,length], [1,1], c='blue', alpha=0.5, linewidth = 3)
        
        # plot the granules on top of the line:
        plt.errorbar(data, y, c='red', xerr=self.Resolution/2, fmt='o', markersize=2, linewidth = 5)
        # note: choose a marker size not bigger than the excluded volume to show it.